# Сapstone project

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'VBIFUSZ1E0GYG0HXKB3NVLBY3IODFCCJKCRMJBGX0VN5QXQN' # your Foursquare ID
CLIENT_SECRET = 'ARVMYNBGADWJ1NDS2CTPHXSKEJOIRCM3ZUB4ISQ3TUE4FJ1G' # your Foursquare Secret
VERSION = '20200227'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VBIFUSZ1E0GYG0HXKB3NVLBY3IODFCCJKCRMJBGX0VN5QXQN
CLIENT_SECRET:ARVMYNBGADWJ1NDS2CTPHXSKEJOIRCM3ZUB4ISQ3TUE4FJ1G


In [102]:
address = 'Blanes'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.6756178 2.7932391


In [103]:
search_query = 'Blanes'
radius = 2000
categories_ID = ['4bf58dd8d48988d1fa931735','4d4b7105d754a06374d81259' , '4bf58dd8d48988d1f9941735',
                 '52f2ab2ebcbc57f1066b8b4f', '4bf58dd8d48988d1fe931735', '4bf58dd8d48988d129951735',
                '4bf58dd8d48988d1ed941735', '4d4b7105d754a06376d81259', '4d4b7105d754a06377d81259','4d4b7104d754a06370d81259' ]
#category_hotel = '4bf58dd8d48988d1fa931735'
#category_Food = '4d4b7105d754a06374d81259'  
category_Food_Drink_Shop = '4bf58dd8d48988d1f9941735'
#category_Bus_Stop = '52f2ab2ebcbc57f1066b8b4f'
#category_Bus Station = '4bf58dd8d48988d1fe931735'
#category_Train_Station = '4bf58dd8d48988d129951735'
#category_Spa = '4bf58dd8d48988d1ed941735'
#category_Nightlife ='4d4b7105d754a06376d81259'
#category_Outdoors_Recreation = '4d4b7105d754a06377d81259'
#category_Arts_Entertainment = '4d4b7104d754a06370d81259'
print(search_query + ' .... OK!')

Blanes .... OK!


In [104]:
frames = []
for category in categories_ID:
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT,category)
    results = requests.get(url).json()
    # assign relevant part of JSON to venues
    venues = results['response']['venues']
    # tranform venues into a dataframe
    if(venues!=[]):
        frames.append(json_normalize(venues))
    else:
        print(category)
    

len(frames)

4bf58dd8d48988d1ed941735


9

In [105]:
dataframe = pd.concat(frames, sort=False)
dataframe

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,59afc101c876c81ac3376f18,Résidence Pierre & Vacances Blanes Playa,"[{'id': '4bf58dd8d48988d12f951735', 'name': 'R...",v-1582918220,False,"Passeig S'Abanell, 32",41.663900,2.784340,"[{'label': 'display', 'lat': 41.6639, 'lng': 2...",1499,17300,ES,Blanes,Cataluña,España,"[Passeig S'Abanell, 32, 17300 Blanes Cataluña,...",NaN,NaN
1,4bd7b885304fce725af133ab,Costa Brava Hotel Blanes,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1582918220,False,"Carrer Anselm Clavé, 48, BAJOS",41.676861,2.788169,"[{'label': 'display', 'lat': 41.676860645, 'ln...",443,17300,ES,Blanes,Cataluña,España,"[Carrer Anselm Clavé, 48, BAJOS, 17300 Blanes ...",NaN,NaN
2,4bd5aad39649ce72d97c511d,Regina Hotel Blanes,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1582918220,False,"Carrer Esperança, 47",41.671456,2.786085,"[{'label': 'display', 'lat': 41.67145578973304...",754,NaN,ES,Blanes,Cataluña,España,"[Carrer Esperança, 47, Blanes Cataluña, España]",NaN,NaN
3,4bd21cc3dce0ce72e4b99f28,Stella Maris Hotel Blanes,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1582918220,False,Avda Vila De Madrid 18,41.662744,2.781239,"[{'label': 'display', 'lat': 41.66274360907374...",1746,NaN,ES,Blanes,Cataluña,España,"[Avda Vila De Madrid 18, Blanes Cataluña, España]",NaN,NaN
4,4bd5aacd9649ce72d47c511d,Beverly Park Hotel Blanes,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1582918220,False,Merce Rodoreda 7,41.660185,2.779914,"[{'label': 'display', 'lat': 41.66018545045686...",2044,NaN,ES,Blanes,Cataluña,España,"[Merce Rodoreda 7, Blanes Cataluña, España]",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,4f1ab578e4b0d1a654df22c0,Agrupacio Fotografica Blanes,"[{'id': '4bf58dd8d48988d1f1931735', 'name': 'G...",v-1582918223,False,Carrer Francesc Macia I Lluisa,41.666429,2.784772,"[{'label': 'display', 'lat': 41.6664290341044,...",1241,17300,ES,Blanes,Cataluña,España,"[Carrer Francesc Macia I Lluisa, 17300 Blanes ...",NaN,NaN
10,5038a022e4b0cc82fd3f580a,Estadi Municipal De Blanes,"[{'id': '4bf58dd8d48988d188941735', 'name': 'S...",v-1582918223,False,NaN,41.668163,2.782049,"[{'label': 'display', 'lat': 41.66816316106323...",1246,NaN,ES,NaN,NaN,España,[España],NaN,NaN
11,53e3e14d498ebc0418a99e45,Hollywood Motor-Show Blanes,"[{'id': '4bf58dd8d48988d1f4931735', 'name': 'R...",v-1582918223,False,NaN,41.668132,2.781708,"[{'label': 'display', 'lat': 41.668132, 'lng':...",1270,NaN,ES,NaN,NaN,España,[España],NaN,NaN
12,4e575a3a483bc10efd92e99b,Bowling Center Blanes,"[{'id': '4bf58dd8d48988d1e4931735', 'name': 'B...",v-1582918223,False,Avinguda d'Europa 41,41.674162,2.776250,"[{'label': 'display', 'lat': 41.67416233, 'lng...",1421,17300,ES,Blanes,Cataluña,España,"[Avinguda d'Europa 41, 17300 Blanes Cataluña, ...",NaN,551261101


In [85]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT,category_Food_Drink_Shop)
url

'https://api.foursquare.com/v2/venues/search?client_id=VBIFUSZ1E0GYG0HXKB3NVLBY3IODFCCJKCRMJBGX0VN5QXQN&client_secret=ARVMYNBGADWJ1NDS2CTPHXSKEJOIRCM3ZUB4ISQ3TUE4FJ1G&ll=41.7197773,2.9312155&v=20200227&query=Tossa de Mar&radius=2000&limit=30&categoryId=52f2ab2ebcbc57f1066b8b46'

In [86]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e5967e0211536001bbebeef'},
 'response': {'venues': []}}

In [110]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.dtypes

id                            object
name                          object
categories                    object
referralId                    object
hasPerk                         bool
location.address              object
location.lat                 float64
location.lng                 float64
location.labeledLatLngs       object
location.distance              int64
location.postalCode           object
location.cc                   object
location.city                 object
location.state                object
location.country              object
location.formattedAddress     object
venuePage.id                  object
dtype: object

In [124]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Ciutat Esportiva Blanes,Stadium,"C. Mas Cuní, 43",41.683731,2.771843,"[{'label': 'display', 'lat': 41.68373094958704...",1995,17300,ES,Blanes,Cataluña,España,"[C. Mas Cuní, 43, 17300 Blanes Cataluña, España]",4da0a4092939b1f7249fe256
1,Teatre de Blanes,Theater,NaN,41.674823,2.791639,"[{'label': 'display', 'lat': 41.6748231766884,...",159,NaN,ES,Blanes,Cataluña,España,"[Blanes Cataluña, España]",4d4330209df9b1f7864e7f94
2,Feria Infantil Blanes,Theme Park,NaN,41.675167,2.795121,"[{'label': 'display', 'lat': 41.67516676579962...",164,NaN,ES,NaN,NaN,España,[España],500c465ae4b0ddfaa76bec79
3,Processó de Blanes,Historic Site,NaN,41.674102,2.792315,"[{'label': 'display', 'lat': 41.67410160778833...",185,NaN,ES,NaN,NaN,España,[España],51560546e4b0e990ef617eed
4,Cavalcada de Blanes,General Entertainment,Muralla 1,41.675227,2.790408,"[{'label': 'display', 'lat': 41.67522745395750...",239,17300,ES,Blanes,Cataluña,España,"[Muralla 1, 17300 Blanes Cataluña, España]",4f05f07ad3e321b78fddc45d
5,Llotja De Peix De Blanes,General Entertainment,NaN,41.675947,2.799859,"[{'label': 'display', 'lat': 41.67594663968653...",551,NaN,ES,NaN,NaN,España,[España],4e32c9557d8bf5432e3fbe4b
6,Bingo Blanes,Casino,NaN,41.678549,2.785171,"[{'label': 'display', 'lat': 41.67854855609123...",745,NaN,ES,Blanes,Cataluña,España,"[Blanes Cataluña, España]",4d751fbe1a8aa35d812f19a7
7,Afic Blanes,General Entertainment,NaN,41.670036,2.787383,"[{'label': 'display', 'lat': 41.67003631591797...",789,NaN,ES,NaN,NaN,España,[España],4fa42718e4b098f42a42d848
8,OCine Blanes,Multiplex,Av. Estación,41.676890,2.773361,"[{'label': 'display', 'lat': 41.67688959889312...",1658,17300,ES,Blanes,Cataluña,España,"[Av. Estación, 17300 Blanes Cataluña, España]",5346fdf7498e495a8b6046f4
9,Agrupacio Fotografica Blanes,General Entertainment,Carrer Francesc Macia I Lluisa,41.666429,2.784772,"[{'label': 'display', 'lat': 41.6664290341044,...",1241,17300,ES,Blanes,Cataluña,España,"[Carrer Francesc Macia I Lluisa, 17300 Blanes ...",4f1ab578e4b0d1a654df22c0


In [125]:
dataframe_filtered.loc[dataframe_filtered['city']=='NaN']

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
